In [ ]:
#configured by Akansh

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import json
import cv2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.metrics import Precision, Recall, AUC

In [ ]:
# Helper to convert to HSV using OpenCV inside tf.data pipeline
def convert_to_hsv(img, space='hsv'):
    img = tf.cast(img, tf.uint8)
    img = tf.numpy_function(lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2HSV), [img], tf.uint8)
    img.set_shape([224, 224, 3])
    return tf.cast(img, tf.float32)

In [ ]:
def preprocess_rgb(image, label):
    def process_single_image(img):
        img = tf.image.random_brightness(img, max_delta=0.15)
        img = tf.image.random_contrast(img, 0.8, 1.2)
        img = tf.image.random_saturation(img, 0.8, 1.2)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.resize_with_crop_or_pad(img, 230, 230)
        img = tf.image.random_crop(img, size=[224, 224, 3])
        return preprocess_input(img)
    
    image = tf.map_fn(process_single_image, image, dtype=tf.float32)
    return image, label


def preprocess_hsv(image, label):
    def process_single_image(img):
        img = convert_to_hsv(img, space='hsv')
        img = tf.image.convert_image_dtype(img, dtype=tf.float32)
        img = tf.image.random_brightness(img, max_delta=0.15)
        img = tf.image.random_contrast(img, 0.8, 1.2)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.resize_with_crop_or_pad(img, 230, 230)
        img = tf.image.random_crop(img, size=[224, 224, 3]) 

        return preprocess_input(img)

    image = tf.map_fn(process_single_image, image, fn_output_signature=tf.float32)
    return image, label

In [ ]:
# Load the validation dataset
validation_set = tf.keras.utils.image_dataset_from_directory(
    r'plant-dataset\val_v1',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    interpolation="bilinear",
    seed=42
)

In [ ]:
# Load the training dataset
training_set = tf.keras.utils.image_dataset_from_directory(
    r'plant-dataset\train_v1',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    interpolation="bilinear",
    seed=42
)

print(training_set.class_names)

In [ ]:
# Load the test dataset
test_set = tf.keras.utils.image_dataset_from_directory(
    r'plant-dataset\test_v1',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=False,
    interpolation="bilinear",
)

In [ ]:
training_set_rgb=training_set.map(preprocess_rgb)
training_set_hsv=training_set.map(preprocess_hsv)

In [ ]:
num_classes = 29 

def build_model(train_base=False):
    # Load ResNet50 model without the top classification layer, using imagenet weights
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    # Freeze the base model layers to prevent them from being trained
    if not train_base:
        base_model.trainable = False
    else:
        for layer in base_model.layers[:-20]:
            layer.trainable = False
        for layer in base_model.layers[-20:]:
            layer.trainable = True
    # Build the new model on top of the ResNet50 base
    model = tf.keras.Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(1024, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=[
            'accuracy',           # Standard accuracy
            Precision(),          # Precision metric
            Recall(),             # Recall metric
            AUC(),                # AUC (Area Under Curve)
            'TruePositives',      # True positive count
            'TrueNegatives',      # True negative count
            'FalsePositives',     # False positive count
            'FalseNegatives'      # False negative count
        ]
    )
    model.summary()
    return model

In [ ]:
# Training the RGB model
model_RGB = build_model()
m_rgb = model_RGB.fit(
    training_set_rgb,
    validation_data=validation_set.map(preprocess_rgb),
    epochs=10
)
model_RGB.save('plant_model_rgb.keras')

# Save training histories to JSON
with open('plant_model_rgb.json', 'w') as f:
    json.dump(m_rgb.history, f)

In [ ]:
# Training the HSV model
model_HSV = build_model(train_base=True)
m_hsv = model_HSV.fit(
    training_set_hsv,
    validation_data=validation_set.map(preprocess_hsv),
    epochs=10
)
model_HSV.save('plant_model_hsv.keras')

# Save training histories to JSON
with open('plant_model_hsv.json', 'w') as f:
    json.dump(m_hsv.history, f)

In [ ]:
# Evaluate models on validation data
results_rgb = model_RGB.evaluate(validation_set.map(preprocess_rgb))
results_hsv = model_HSV.evaluate(validation_set.map(preprocess_hsv))

print(f"RGB Model Evaluation Results: {results_rgb}")
print(f"HSV Model Evaluation Results: {results_hsv}")

# Save evaluation results
with open('plant_model_rgb_evaluation.json', 'w') as f:
    json.dump(results_rgb, f)

with open('plant_model_hsv_evaluation.json', 'w') as f:
    json.dump(results_hsv, f)

In [ ]:
# # Evaluate the model on the training set
# results = model.evaluate(training_set)

# # Extracting loss and accuracy along with additional metrics
# train_loss = results[0]
# train_acc = results[1]
# precision = results[2]
# recall = results[3]
# auc = results[4]
# true_positives = results[5]
# true_negatives = results[6]
# false_positives = results[7]
# false_negatives = results[8]

# # Printing the results
# print(f'Train Loss: {train_loss}, Train Accuracy: {train_acc}')
# print(f'Precision: {precision}, Recall: {recall}, AUC: {auc}')
# print(f'True Positives: {true_positives}, True Negatives: {true_negatives}')
# print(f'False Positives: {false_positives}, False Negatives: {false_negatives}')

In [ ]:
# # Evaluate the model on the validation set
# results = model.evaluate(validation_set)

# # Extracting loss and accuracy along with additional metrics
# val_loss = results[0]
# val_acc = results[1]
# precision = results[2]
# recall = results[3]
# auc = results[4]
# true_positives = results[5]
# true_negatives = results[6]
# false_positives = results[7]
# false_negatives = results[8]

# # Printing the results
# print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_acc}')
# print(f'Precision: {precision}, Recall: {recall}, AUC: {auc}')
# print(f'True Positives: {true_positives}, True Negatives: {true_negatives}')
# print(f'False Positives: {false_positives}, False Negatives: {false_negatives}')